In [1]:
import QpixAsicArray as qparray
from QpixAsic import PixelHit, ProcQueue
import random
import numpy as np
random.seed(2)
qpa = qparray.QpixAsicArray(16, 16, debug=0.0)

In [6]:
timeNow = 1.
timeEnd = 4.0
tickNow = 50e6
deltaT = 1.0
deltaTick = 50e6
timeEpsilon = 1e-6
eventTimes = []
hitsPerEvent, eventTimes = [], []
procQueue = ProcQueue()
    
while(timeNow < timeEnd):

    items = 0
    topProc = []
    midProc = []
    botProc = []
    lenRecv = []
    asicRow = []
    asicCol = []

    for asic in qpa:
        newProcessItems = asic.Process(timeNow - timeEpsilon)
        if newProcessItems:
            print("WARNING: ASIC had things left to do at next maor time step")

    procQueue.AddQueueItem(qpa[0][0], 3, PixelHit(tickNow, [], None, None), timeNow)

    while(procQueue.Length() > 0):

        nextItem = procQueue.PopQueue()
        items += 1

        topProc.append(qpa.ProcessArray(procQueue, nextItem.absTime))

        newProcessItems = nextItem.asic.ReceiveData(nextItem)
        if newProcessItems:
            midProc.append(1)
            lenRecv.append(len(newProcessItems))
            for item in newProcessItems:
                procQueue.AddQueueItem(*item)
        else:
            midProc.append(0)
            lenRecv.append(0)

        botProc.append(qpa.ProcessArray(procQueue, nextItem.absTime))

    eventTimes.append(qpa._daqNode.absTimeNow-timeNow)
    hitsPerEvent.append(qpa._daqNode.daqHits)
    qpa._daqNode.daqHits = 0
    timeNow += deltaT
    tickNow += deltaTick
    print("ran through this many items:", items)
    print('topProc:', np.sum(topProc), len(topProc), topProc[:20])
    print('midProc:  ', np.sum(midProc), len(midProc), midProc[:20])
    print('botProc:  ', np.sum(botProc), len(botProc), botProc[:20])
    print('lenRecv:  ', np.sum(lenRecv), len(lenRecv), lenRecv[:20])
    print('sum:', items-np.sum(topProc+midProc+botProc))

ran through this many items: 1
topProc: 0 1 [0]
midProc:   0 1 [0]
botProc:   0 1 [0]
lenRecv:   0 1 [0]
sum: 1
ran through this many items: 1
topProc: 0 1 [0]
midProc:   0 1 [0]
botProc:   0 1 [0]
lenRecv:   0 1 [0]
sum: 1
ran through this many items: 1
topProc: 0 1 [0]
midProc:   0 1 [0]
botProc:   0 1 [0]
lenRecv:   0 1 [0]
sum: 1


In [3]:
print("MAX QUEUE DEPTHS")
# Check the current queue depths
for asic in qpa[10][:3]:
    print(asic.row, asic.col, " - local: "+str(asic._maxLocalDepth), f"- measures {asic._measurements}", f"- transactions {asic._transmissions}", f"- receptions {asic._receptions}", end='\n\t\t')
    print(f"{asic.maxConnDepths[0]}\n\t{asic.maxConnDepths[3]}", end='\t\t')
    print(f"{asic.maxConnDepths[1]}\n\t\t{asic.maxConnDepths[2]}", end='\n')
    # for d in range(0,4):
    #   print(str(asic.maxConnDepths[d])+", ",end='')
    # print()

print("PROCESSING TIMES")
for i in range(0,len(eventTimes)):
  print(str(eventTimes[i]))

print("HITS PER EVENT")
for i in range(0,len(hitsPerEvent)):
  print(str(hitsPerEvent[i]))

MAX QUEUE DEPTHS
10 0  - local: 3 - measures 3 - transactions 49 - receptions 39
		0
	0		0
		2
10 1  - local: 4 - measures 3 - transactions 295 - receptions 282
		0
	0		2
		41
10 2  - local: 4 - measures 3 - transactions 128 - receptions 115
		0
	0		0
		3
PROCESSING TIMES
0.003346953260890295
0.0033909665908371522
0.0034225906228790848
HITS PER EVENT
707
715
723


In [4]:
import numpy as np
transactions, measures = [], []
for asic in qpa:
    transactions.append(asic._transmissions)
    measures.append(asic._measurements)
    # print(asic)
print("transaction count:", np.sum(transactions))
print("measure count:", np.sum(measures))
print("difference from total:", items-(np.sum(measures)+np.sum(transactions)))

transaction count: 34981
measure count: 768
difference from total: -23664
